# Thesis

"Forecasting NBA match results using multivariate Generalized Autoregressive Score Models"

Travis van Cornewal\
(VU Student Number: 2731231)

supervisor: dr. Janneke van Brummelen\
co-reader: dr. Jan Bauer

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import nbinom, poisson
from scipy.optimize import minimize

from functions import GASNegBin, GASNegBinIntercept, GASPoisson, GASPoissonIntercept, run_gas_forecast_loop, compute_toto_probs, diebold_mariano

In [2]:
data = pd.read_csv("nba_game.csv")
data = data[data['season_type'].isin(['Regular Season', 'Pre Season'])]
data['game_date'] = pd.to_datetime(data['game_date'], errors='coerce')
data = data[['game_date', 'team_name_home', 'team_name_away', 'pts_home', 'pts_away', 'season_type']]
data = data.dropna()
data['pts_home'] = data['pts_home'].astype(int)
data['pts_away'] = data['pts_away'].astype(int)

In [3]:
def assign_season(date):
    return f"{date.year}/{date.year + 1}" if date.month >= 7 else f"{date.year - 1}/{date.year}"

def is_weekend(date):
    return date.weekday() >= 5  # Saturday = 5, Sunday = 6

def assign_rounds(season_df):
    season_df = season_df.sort_values("game_date").copy()
    round_counter = 1
    last_type = None
    last_date = None
    round_ids = []

    for date in season_df['game_date']:
        current_type = 'weekend' if is_weekend(date) else 'midweek'
        if last_date is not None:
            days_apart = (date - last_date).days
            if days_apart > 2 or current_type != last_type:
                round_counter += 1

        round_ids.append(round_counter)
        last_date = date
        last_type = current_type

    season_df['round'] = round_ids
    return season_df

In [4]:
data = data[(data['game_date'] >= "2013-07-01") & (data['game_date'] < "2023-06-27")].reset_index() 

# Season
data['season'] = data['game_date'].apply(assign_season)

# Season number
season_order = sorted(data['season'].unique())
season_to_number = {season: i + 1 for i, season in enumerate(season_order)}
data['season_number'] = data['season'].map(season_to_number)

# Assign rounds per season
data = data.groupby('season', group_keys=False).apply(assign_rounds).sort_values('game_date')

In [5]:
# Apply mapping
team_name_mapping = {
    "LA Clippers": "Los Angeles Clippers",
    "Charlotte Hornets": "Charlotte Bobcats"  
}
data['team_name_home'] = data['team_name_home'].replace(team_name_mapping)
data['team_name_away'] = data['team_name_away'].replace(team_name_mapping)

In [6]:
init_data = data[data['season'] == "2013/2014"]
train_test_data = data[data['season'].isin([
    "2014/2015", "2015/2016", "2016/2017", "2017/2018",
    "2018/2019", "2019/2020", "2020/2021", "2021/2022", "2022/2023"
])]
train_data = train_test_data[train_test_data['season'].isin(["2014/2015", "2015/2016", "2016/2017", "2017/2018"])]
test_data = train_test_data[train_test_data['season'].isin(["2018/2019", "2019/2020", "2020/2021", "2021/2022", "2022/2023"])]

In [7]:
all_teams = sorted(
    set(init_data['team_name_home']).union(init_data['team_name_away']) |
    set(train_data['team_name_home']).union(train_data['team_name_away']) |
    set(test_data['team_name_home']).union(test_data['team_name_away'])
)
team_idx = {team: i for i, team in enumerate(all_teams)}
N = len(all_teams)

init_teams = sorted(set(init_data['team_name_home']) | set(init_data['team_name_away']))
init_team_idx = {team: i for i, team in enumerate(init_teams)}
M = len(init_teams)

In [8]:
data

,index,game_date,team_name_home,team_name_away,pts_home,pts_away,season_type,season,season_number,round
1230,53388,2013-10-05,Houston Rockets,New Orleans Pelicans,115,116,Pre Season,2013/2014,1,1
1231,53389,2013-10-05,Istanbul Fenerbahce Ulker,Oklahoma City Thunder,82,95,Pre Season,2013/2014,1,1
1232,53390,2013-10-05,Indiana Pacers,Chicago Bulls,76,82,Pre Season,2013/2014,1,1
1233,53391,2013-10-05,Los Angeles Lakers,Golden State Warriors,104,95,Pre Season,2013/2014,1,1
1234,53392,2013-10-06,Bilbao Basket,Philadelphia 76ers,104,106,Pre Season,2013/2014,1,1
...,...,...,...,...,...,...,...,...,...,...
12600,65527,2023-04-09,Minnesota Timberwolves,New Orleans Pelicans,113,108,Regular Season,2022/2023,10,54
12613,65540,2023-04-09,Toronto Raptors,Milwaukee Bucks,121,105,Regular Season,2022/2023,10,54
12614,65541,2023-04-09,Cleveland Cavaliers,Charlotte Bobcats,95,106,Regular Season,2022/2023,10,54
12605,65532,2023-04-09,Los Angeles Lakers,Utah Jazz,128,117,Regular Season,2022/2023,10,54


## Negative Binomial with intercept

In [9]:
init_data_np = init_data[
    init_data['team_name_home'].isin(init_team_idx)
    & init_data['team_name_away'].isin(init_team_idx)
].copy()

init_data_np['i'] = init_data_np['team_name_home'].map(init_team_idx)
init_data_np['j'] = init_data_np['team_name_away'].map(init_team_idx)

x_vals = init_data_np['pts_home'].values
y_vals = init_data_np['pts_away'].values
i_vals = init_data_np['i'].values
j_vals = init_data_np['j'].values

def log_likelihood(params):
    alpha = params[:M]
    beta = params[M:2 * M]
    delta = params[-3]
    intercept = params[-2]
    log_r = params[-1]
    r = np.exp(log_r)

    mu_home = np.exp(intercept + delta + alpha[i_vals] - beta[j_vals])
    mu_away = np.exp(intercept + alpha[j_vals] - beta[i_vals])

    logpmf_home = nbinom.logpmf(x_vals, r, r / (r + mu_home))
    logpmf_away = nbinom.logpmf(y_vals, r, r / (r + mu_away))

    return -np.sum(logpmf_home + logpmf_away)

def alpha_sum_constraint(params):
    return np.sum(params[:M])

def beta_sum_constraint(params):
    return np.sum(params[M:2 * M])

constraints = [
    {'type': 'eq', 'fun': alpha_sum_constraint},
    {'type': 'eq', 'fun': beta_sum_constraint}
]

params_init = np.zeros(2 * M + 3)
params_init[-3] = 0.04            # delta
params_init[-2] = 3.5             # intercept
params_init[-1] = np.log(250)     # log(r)

result = minimize(
    log_likelihood,
    x0=params_init,
    method='SLSQP',
    constraints=constraints,
    options={'disp': False, 'ftol': 1e-10, 'eps': 1e-8, 'maxiter': 5000}
)

print("\n==== Optimization Results ====")
if result.success:
    print("Optimization converged successfully.")
else:
    print("Optimization failed to converge.")

params = result.x
alpha, beta = params[:M], params[M:2 * M]
delta = params[-3]
intercept = params[-2]
log_r = params[-1]
r = np.exp(log_r)

# === Map alpha and beta to full vector ===
alpha_full = np.zeros(N)
beta_full = np.zeros(N)

for team in all_teams:
    idx = team_idx[team]
    if team in init_team_idx:
        j = init_team_idx[team]
        alpha_full[idx] = alpha[j]
        beta_full[idx] = beta[j]

f1_estimate_nb = np.concatenate([alpha_full, beta_full])

print("Finished initialization")
print(f"delta (home advantage): {delta:.4f}")
print(f"intercept: {intercept:.4f}")
print(f"log_r (dispersion): {log_r:.4f}")
print(f"r (dispersion): {r:.4f}")
print(f"mean alpha: {np.mean(alpha):.4f}, mean beta: {np.mean(beta):.4f}")

C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\4136831700.py:22: RuntimeWarning: overflow encountered in exp
  mu_home = np.exp(intercept + delta + alpha[i_vals] - beta[j_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\4136831700.py:23: RuntimeWarning: overflow encountered in exp
  mu_away = np.exp(intercept + alpha[j_vals] - beta[i_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\4136831700.py:25: RuntimeWarning: invalid value encountered in divide
  logpmf_home = nbinom.logpmf(x_vals, r, r / (r + mu_home))
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\4136831700.py:26: RuntimeWarning: invalid value encountered in divide
  logpmf_away = nbinom.logpmf(y_vals, r, r / (r + mu_away))



==== Optimization Results ====
Optimization converged successfully.
Finished initialization
delta (home advantage): 0.0245
intercept: 4.5826
log_r (dispersion): 7.0599
r (dispersion): 1164.3799
mean alpha: 0.0000, mean beta: 0.0000


In [10]:
initial_strengths_nb = pd.DataFrame({
    "team": all_teams,
    "initial attack strength": f1_estimate_nb[:N],
    "initial defense strength": f1_estimate_nb[N:]
}).sort_values(by="initial attack strength", ascending=False)
initial_strengths_nb

,team,initial attack strength,initial defense strength
20,Houston Rockets,0.085670,-0.016532
24,Los Angeles Clippers,0.075561,0.007113
32,Minnesota Timberwolves,0.073804,-0.027199
40,Portland Trail Blazers,0.066340,-0.006483
36,Oklahoma City Thunder,0.060219,0.016087
39,Phoenix Suns,0.057022,-0.008565
43,San Antonio Spurs,0.057013,0.029890
13,Dallas Mavericks,0.048397,-0.006595
14,Denver Nuggets,0.045498,-0.044241
17,Golden State Warriors,0.038524,0.022115


In [11]:
model = GASNegBinIntercept(all_teams, f1_estimate=f1_estimate_nb)
model.fit(train_data)


==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9936, b2=0.9972
delta=0.0251, intercept=4.5796, r=1838.6608
Optimization converged successfully.


In [12]:
forecast_results_nb, latent_history_nb = run_gas_forecast_loop(
    train_test_data=train_test_data,
    test_data=test_data,
    all_teams=all_teams,
    f1_estimate=f1_estimate_nb,
    model_class=GASNegBinIntercept,
    compute_probs_fn=compute_toto_probs,
    distr='negbin'
)


=== Training for season 2018/2019 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9936, b2=0.9972
delta=0.0251, intercept=4.5796, r=1838.6608
Optimization converged successfully.

=== Forecasting for season 2018/2019 ===

=== Training for season 2019/2020 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9947, b2=0.9982
delta=0.0251, intercept=4.5802, r=1841.0022
Optimization converged successfully.

=== Forecasting for season 2019/2020 ===

=== Training for season 2020/2021 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9938, b2=0.9983
delta=0.0246, intercept=4.5826, r=1848.5394
Optimization converged successfully.

=== Forecasting for season 2020/2021 ===

=== Training for season 2021/2022 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9954, b2=0.9984
delta=0.0222, intercept=4.5808, r=1841.5924
Optimization converged successfully.

=== Forecasting for season 2021/2022 ===

=== Training for season 2022/2023 ===

==== Optimizatio

In [13]:
forecast_df_nb = pd.DataFrame(forecast_results_nb)
latent_df_nb = pd.DataFrame([
    {
        "team": team,
        "date": entry["date"],
        "attack": entry["attack"],
        "defense": entry["defense"],
        "season_type": entry["season_type"]
    }
    for team, entries in latent_history_nb.items()
    for entry in entries
])

## Negative Binomial static

In [14]:
forecast_results_nb_static = []
latent_history_nb_static = defaultdict(list)
season_groups = test_data.groupby("season")

for season, season_data in season_groups:
    print(f"\n=== Training for season {season} ===")
    season_start_date = season_data["game_date"].min()

    history_data = train_test_data[
        (train_test_data["game_date"] < season_start_date) &
        (train_test_data["season_type"].isin(["Regular Season", "Pre Season"]))
    ]

    model_static = GASNegBinIntercept(all_teams, f1_estimate=f1_estimate_nb)
    model_static.fit(history_data)
    f_static = model_static.f.copy()
    all_games = season_data[season_data["season_type"].isin(["Pre Season", "Regular Season"])]

    print(f"\n=== Forecasting for season {season} ===")
    for forecast_date in sorted(all_games["game_date"].unique()):
        games_today = all_games[all_games["game_date"] == forecast_date]

        for row in games_today.itertuples():
            i = model_static.team_idx.get(row.team_name_home)
            j = model_static.team_idx.get(row.team_name_away)
            if i is None or j is None:
                continue

            fij = model_static._get_f_vector(i, j, f_static)
            mu_home, mu_away = model_static._means(fij)
            p_home_win, p_away_win, _ = compute_toto_probs(mu_home=mu_home, mu_away=mu_away, distr='negbin', r=model_static.r)

            true_vec = [1, 0] if row.pts_home > row.pts_away else [0, 1]
            pred_vec = [p_home_win, p_away_win]
            rps = 0.5 * np.sum((np.cumsum(pred_vec) - np.cumsum(true_vec)) ** 2)
            log_score = -np.log(p_home_win if row.pts_home > row.pts_away else p_away_win)

            forecast_results_nb_static.append({
                "date": forecast_date,
                "home_team": row.team_name_home,
                "away_team": row.team_name_away,
                "true_home": row.pts_home,
                "true_away": row.pts_away,
                "mu_home": mu_home,
                "mu_away": mu_away,
                "p_home_win": p_home_win,
                "p_away_win": p_away_win,
                "rps": rps,
                "log_score": log_score,
                "season_type": row.season_type,
                "model": "static"
            })

        for team in all_teams:
            idx = model_static.team_idx[team]
            latent_history_nb_static[team].append({
                "team_idx": idx,
                "date": forecast_date,
                "attack": f_static[idx],
                "defense": f_static[model_static.N + idx],
                "season_type": "Static"
            })

forecast_df_nb_static = pd.DataFrame(forecast_results_nb_static)


=== Training for season 2018/2019 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9936, b2=0.9972
delta=0.0251, intercept=4.5796, r=1838.6608
Optimization converged successfully.

=== Forecasting for season 2018/2019 ===

=== Training for season 2019/2020 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9947, b2=0.9982
delta=0.0251, intercept=4.5802, r=1841.0022
Optimization converged successfully.

=== Forecasting for season 2019/2020 ===

=== Training for season 2020/2021 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9938, b2=0.9983
delta=0.0246, intercept=4.5826, r=1848.5394
Optimization converged successfully.

=== Forecasting for season 2020/2021 ===

=== Training for season 2021/2022 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9954, b2=0.9984
delta=0.0222, intercept=4.5808, r=1841.5924
Optimization converged successfully.

=== Forecasting for season 2021/2022 ===

=== Training for season 2022/2023 ===

==== Optimizatio

## Poisson with intercept

In [15]:
init_data_np = init_data[
    init_data['team_name_home'].isin(init_team_idx)
    & init_data['team_name_away'].isin(init_team_idx)
].copy()

init_data_np['i'] = init_data_np['team_name_home'].map(init_team_idx)
init_data_np['j'] = init_data_np['team_name_away'].map(init_team_idx)

x_vals = init_data_np['pts_home'].values
y_vals = init_data_np['pts_away'].values
i_vals = init_data_np['i'].values
j_vals = init_data_np['j'].values

M = len(init_team_idx)  
N = len(all_teams)     

def log_likelihood(params):
    alpha = params[:M]
    beta = params[M:2 * M]
    delta = params[-2]
    intercept = params[-1]

    mu_home = np.exp(intercept + delta + alpha[i_vals] - beta[j_vals])
    mu_away = np.exp(intercept + alpha[j_vals] - beta[i_vals])

    logpmf_home = poisson.logpmf(x_vals, mu_home)
    logpmf_away = poisson.logpmf(y_vals, mu_away)

    return -np.sum(logpmf_home + logpmf_away)

def alpha_sum_constraint(params):
    return np.sum(params[:M])

def beta_sum_constraint(params):
    return np.sum(params[M:2 * M])

constraints = [
    {'type': 'eq', 'fun': alpha_sum_constraint},
    {'type': 'eq', 'fun': beta_sum_constraint}
]

params_init = np.zeros(2 * M + 2)
params_init[-2] = 0.04   # delta
params_init[-1] = 3.5    # intercept

result = minimize(
    log_likelihood,
    x0=params_init,
    method='SLSQP',
    constraints=constraints,
    options={'disp': False, 'ftol': 1e-10, 'eps': 1e-8, 'maxiter': 5000}
)

print("\n==== Optimization Results ====")
if result.success:
    print("Optimization converged successfully.")
else:
    print("Optimization failed to converge.")

params = result.x
alpha, beta = params[:M], params[M:2 * M]
delta = params[-2]
intercept = params[-1]

alpha_full = np.zeros(N)
beta_full = np.zeros(N)

for team in all_teams:
    idx = team_idx[team]
    if team in init_team_idx:
        j = init_team_idx[team]
        alpha_full[idx] = alpha[j]
        beta_full[idx] = beta[j]

f1_estimate_poisson = np.concatenate([alpha_full, beta_full])

print(f"delta (home advantage): {delta:.4f}")
print(f"intercept: {intercept:.4f}")
print(f"mean alpha: {np.mean(alpha):.4f}, mean beta: {np.mean(beta):.4f}")
print("Finished initialization")

C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1208774360.py:23: RuntimeWarning: overflow encountered in exp
  mu_home = np.exp(intercept + delta + alpha[i_vals] - beta[j_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1208774360.py:24: RuntimeWarning: overflow encountered in exp
  mu_away = np.exp(intercept + alpha[j_vals] - beta[i_vals])
c:\Users\tcornew\AppData\Local\anaconda3\Lib\site-packages\scipy\stats\_discrete_distns.py:860: RuntimeWarning: invalid value encountered in subtract
  Pk = special.xlogy(k, mu) - gamln(k + 1) - mu



==== Optimization Results ====
Optimization converged successfully.
delta (home advantage): 0.0245
intercept: 4.5826
mean alpha: -0.0000, mean beta: 0.0000
Finished initialization


In [16]:
model = GASPoissonIntercept(all_teams, f1_estimate=f1_estimate_poisson)
model.fit(train_data)


==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9934, b2=0.9971
delta=0.0250, intercept=4.5835
Optimization converged successfully.


In [17]:
forecast_results_poisson, latent_history_poisson = run_gas_forecast_loop(
    train_test_data=train_test_data,
    test_data=test_data,
    all_teams=all_teams,
    f1_estimate=f1_estimate_poisson,
    model_class=GASPoissonIntercept,
    compute_probs_fn=compute_toto_probs,
    distr='poisson'
)


=== Training for season 2018/2019 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9934, b2=0.9971
delta=0.0250, intercept=4.5835
Optimization converged successfully.

=== Forecasting for season 2018/2019 ===

=== Training for season 2019/2020 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9943, b2=0.9979
delta=0.0250, intercept=4.5858
Optimization converged successfully.

=== Forecasting for season 2019/2020 ===

=== Training for season 2020/2021 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9942, b2=0.9980
delta=0.0246, intercept=4.5880
Optimization converged successfully.

=== Forecasting for season 2020/2021 ===

=== Training for season 2021/2022 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9952, b2=0.9983
delta=0.0225, intercept=4.5854
Optimization converged successfully.

=== Forecasting for season 2021/2022 ===

=== Training for season 2022/2023 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9959, b2=0.

In [18]:
forecast_df_poisson = pd.DataFrame(forecast_results_poisson)

## Poisson static 

In [19]:
forecast_results_poisson_static = []
latent_history_poisson_static = defaultdict(list)

season_groups = test_data.groupby("season")

for season, season_data in season_groups:
    print(f"\n=== STATIC Forecasting for season {season} ===")
    season_start_date = season_data["game_date"].min()

    history_data = train_test_data[
        (train_test_data["game_date"] < season_start_date) &
        (train_test_data["season_type"].isin(["Regular Season", "Pre Season"]))
    ]

    model_static = GASPoissonIntercept(all_teams, f1_estimate=f1_estimate_poisson)
    model_static.fit(history_data)
    f_static = model_static.f.copy()

    all_games = season_data[season_data["season_type"].isin(["Pre Season", "Regular Season"])]

    for forecast_date in sorted(all_games["game_date"].unique()):
        print(f"  Forecasting (static) {forecast_date}")
        games_today = all_games[all_games["game_date"] == forecast_date]
        for row in games_today.itertuples():
            i = model_static.team_idx.get(row.team_name_home)
            j = model_static.team_idx.get(row.team_name_away)
            if i is None or j is None:
                continue

            fij = model_static._get_f_vector(i, j, f_static)
            mu_home, mu_away = model_static._means(fij)
            p_home_win, p_away_win, _ = compute_toto_probs(mu_home=mu_home, mu_away=mu_away, distr='poisson')

            true_vec = [1, 0] if row.pts_home > row.pts_away else [0, 1]
            pred_vec = [p_home_win, p_away_win]
            rps = 0.5 * np.sum((np.cumsum(pred_vec) - np.cumsum(true_vec)) ** 2)
            log_score = -np.log(p_home_win if row.pts_home > row.pts_away else p_away_win)

            forecast_results_poisson_static.append({
                "date": forecast_date,
                "home_team": row.team_name_home,
                "away_team": row.team_name_away,
                "true_home": row.pts_home,
                "true_away": row.pts_away,
                "mu_home": mu_home,
                "mu_away": mu_away,
                "p_home_win": p_home_win,
                "p_away_win": p_away_win,
                "rps": rps,
                "log_score": log_score,
                "season_type": row.season_type,
                "model": "static"
            })

        for team in all_teams:
            idx = model_static.team_idx[team]
            latent_history_poisson_static[team].append({
                "team_idx": idx,
                "date": forecast_date,
                "attack": f_static[idx],
                "defense": f_static[model_static.N + idx],
                "season_type": "Static"
            })

forecast_df_poisson_static = pd.DataFrame(forecast_results_poisson_static)


=== STATIC Forecasting for season 2018/2019 ===

==== Optimization Results ====
a1=0.0005, a2=0.0004
b1=0.9934, b2=0.9971
delta=0.0250, intercept=4.5835
Optimization converged successfully.
  Forecasting (static) 2018-10-16T00:00:00.000000000
  Forecasting (static) 2018-10-17T00:00:00.000000000
  Forecasting (static) 2018-10-18T00:00:00.000000000
  Forecasting (static) 2018-10-19T00:00:00.000000000
  Forecasting (static) 2018-10-20T00:00:00.000000000
  Forecasting (static) 2018-10-21T00:00:00.000000000
  Forecasting (static) 2018-10-22T00:00:00.000000000
  Forecasting (static) 2018-10-23T00:00:00.000000000
  Forecasting (static) 2018-10-24T00:00:00.000000000
  Forecasting (static) 2018-10-25T00:00:00.000000000
  Forecasting (static) 2018-10-26T00:00:00.000000000
  Forecasting (static) 2018-10-27T00:00:00.000000000
  Forecasting (static) 2018-10-28T00:00:00.000000000
  Forecasting (static) 2018-10-29T00:00:00.000000000
  Forecasting (static) 2018-10-30T00:00:00.000000000
  Forecasting 

## Negative Binomial no intercept

In [20]:
init_data_np = init_data[
    init_data['team_name_home'].isin(init_team_idx)
    & init_data['team_name_away'].isin(init_team_idx)
].copy()
init_data_np['i'] = init_data_np['team_name_home'].map(init_team_idx)
init_data_np['j'] = init_data_np['team_name_away'].map(init_team_idx)

x_vals = init_data_np['pts_home'].values
y_vals = init_data_np['pts_away'].values
i_vals = init_data_np['i'].values
j_vals = init_data_np['j'].values

def log_likelihood(params):
    alpha = params[:M]
    beta = params[M:2 * M]
    delta = params[-2]
    log_r = params[-1]
    r = np.exp(log_r)

    mu_home = np.exp(delta + alpha[i_vals] - beta[j_vals])
    mu_away = np.exp(alpha[j_vals] - beta[i_vals])

    logpmf_home = nbinom.logpmf(x_vals, r, r / (r + mu_home))
    logpmf_away = nbinom.logpmf(y_vals, r, r / (r + mu_away))

    return -np.sum(logpmf_home + logpmf_away)

def alpha_sum_constraint(params):
    return np.sum(params[:M])

constraints = [{'type': 'eq', 'fun': alpha_sum_constraint}]

params_init = np.zeros(2 * M + 2)
params_init[-2] = 0.04            # delta
params_init[-1] = np.log(250)     # log(r)

result = minimize(
    log_likelihood,
    x0=params_init,
    method='SLSQP',
    constraints=constraints,
    options={'disp': False, 'ftol': 1e-10, 'eps': 1e-8, 'maxiter': 5000}
)

print("\n==== Optimization Results ====")
if result.success:
    print("Optimization converged successfully.")
else:
    print("Optimization failed to converge.")

params = result.x
alpha, beta = params[:M], params[M:2 * M]
delta = params[-2]
log_r = params[-1]
r = np.exp(log_r)

alpha_full = np.zeros(N)
beta_full = np.zeros(N)

for team in all_teams:
    idx = team_idx[team]
    if team in init_team_idx:
        j = init_team_idx[team]
        alpha_full[idx] = alpha[j]
        beta_full[idx] = beta[j]
    else: 
        alpha_full[idx] = 0.0
        beta_full[idx] = np.mean(beta)

f1_estimate_nb_no_intercept = np.concatenate([alpha_full, beta_full])

print(f"delta (home advantage): {delta:.4f}")
print(f"log_r (dispersion): {log_r:.4f}")
print(f"r (dispersion): {r:.4f}")
print(f"mean alpha: {np.mean(alpha):.4f}, mean beta: {np.mean(beta):.4f}")
print("Finished initialization (no intercept)")

C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1043792581.py:20: RuntimeWarning: overflow encountered in exp
  mu_home = np.exp(delta + alpha[i_vals] - beta[j_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1043792581.py:21: RuntimeWarning: overflow encountered in exp
  mu_away = np.exp(alpha[j_vals] - beta[i_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1043792581.py:18: RuntimeWarning: overflow encountered in exp
  r = np.exp(log_r)
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1043792581.py:23: RuntimeWarning: invalid value encountered in divide
  logpmf_home = nbinom.logpmf(x_vals, r, r / (r + mu_home))
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\1043792581.py:24: RuntimeWarning: invalid value encountered in divide
  logpmf_away = nbinom.logpmf(y_vals, r, r / (r + mu_away))



==== Optimization Results ====
Optimization converged successfully.
delta (home advantage): 0.0245
log_r (dispersion): 7.0753
r (dispersion): 1182.3492
mean alpha: -0.0000, mean beta: -4.5826
Finished initialization (no intercept)


In [21]:
forecast_results_nb_no_intercept, latent_history_nb_no_intercept  = run_gas_forecast_loop(
    train_test_data=train_test_data,
    test_data=test_data,
    all_teams=all_teams,
    f1_estimate=f1_estimate_nb_no_intercept,
    model_class=GASNegBin,
    compute_probs_fn=compute_toto_probs,
    distr='negbin'
)


=== Training for season 2018/2019 ===
==== Optimization Results ====
a1=0.0006, a2=0.0004
b1=0.9931, b2=0.9970
delta=0.0251, r=576.0426
Optimization converged successfully.

=== Forecasting for season 2018/2019 ===

=== Training for season 2019/2020 ===
==== Optimization Results ====
a1=0.0006, a2=0.0004
b1=0.9942, b2=0.9981
delta=0.0252, r=548.4272
Optimization converged successfully.

=== Forecasting for season 2019/2020 ===

=== Training for season 2020/2021 ===
==== Optimization Results ====
a1=0.0006, a2=0.0004
b1=0.9939, b2=0.9983
delta=0.0245, r=546.8963
Optimization converged successfully.

=== Forecasting for season 2020/2021 ===

=== Training for season 2021/2022 ===


c:\Users\tcornew\Downloads\thesis\code\m3\final\final\functions.py:165: RuntimeWarning: divide by zero encountered in log
  return gammaln(x + r) - gammaln(r) - gammaln(x + 1) + r * np.log(p) + x * np.log(1 - p)
c:\Users\tcornew\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


==== Optimization Results ====
a1=0.0006, a2=0.0004
b1=0.9953, b2=0.9984
delta=0.0223, r=528.4602
Optimization converged successfully.

=== Forecasting for season 2021/2022 ===

=== Training for season 2022/2023 ===
==== Optimization Results ====
a1=0.0006, a2=0.0005
b1=0.9961, b2=0.9980
delta=0.0214, r=513.5504
Optimization converged successfully.

=== Forecasting for season 2022/2023 ===
Finished forecasting.


In [22]:
forecast_df_nb_no_intercept = pd.DataFrame(forecast_results_nb_no_intercept)

## Poisson no intercept

In [23]:
init_data_np = init_data[
    init_data['team_name_home'].isin(init_team_idx)
    & init_data['team_name_away'].isin(init_team_idx)
].copy()

init_data_np['i'] = init_data_np['team_name_home'].map(init_team_idx)
init_data_np['j'] = init_data_np['team_name_away'].map(init_team_idx)

x_vals = init_data_np['pts_home'].values
y_vals = init_data_np['pts_away'].values
i_vals = init_data_np['i'].values
j_vals = init_data_np['j'].values

M = len(init_team_idx)  
N = len(all_teams)

def log_likelihood(params):
    alpha = params[:M]
    beta = params[M:2 * M]
    delta = params[-1]

    mu_home = np.exp(delta + alpha[i_vals] - beta[j_vals])
    mu_away = np.exp(alpha[j_vals] - beta[i_vals])

    logpmf_home = poisson.logpmf(x_vals, mu_home)
    logpmf_away = poisson.logpmf(y_vals, mu_away)

    return -np.sum(logpmf_home + logpmf_away)

def alpha_sum_constraint(params):
    return np.sum(params[:M])

constraints = [{'type': 'eq', 'fun': alpha_sum_constraint}]

params_init = np.zeros(2 * M + 1)
params_init[-1] = 0.04  # delta only

result = minimize(
    log_likelihood,
    x0=params_init,
    method='SLSQP',
    constraints=constraints,
    options={'disp': False, 'ftol': 1e-10, 'eps': 1e-8, 'maxiter': 5000}
)

print("\n==== Optimization Results ====")
if result.success:
    print("Optimization converged successfully.")
else:
    print("Optimization failed to converge.")

params = result.x
alpha, beta = params[:M], params[M:2 * M]
delta = params[-1]

alpha_full = np.zeros(N)
beta_full = np.zeros(N)
mean_beta = np.mean(beta)

for team in all_teams:
    idx = team_idx[team]
    if team in init_team_idx:
        j = init_team_idx[team]
        alpha_full[idx] = alpha[j]
        beta_full[idx] = beta[j]
    else:
        alpha_full[idx] = 0.0
        beta_full[idx] = 0.0

f1_estimate_poisson_no_intercept = np.concatenate([alpha_full, beta_full])

print(f"delta (home advantage): {delta:.4f}")
print(f"mean alpha: {np.mean(alpha):.4f}, mean beta: {np.mean(beta):.4f}")
print("Finished initialization")

C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\3996712011.py:22: RuntimeWarning: overflow encountered in exp
  mu_home = np.exp(delta + alpha[i_vals] - beta[j_vals])
C:\Users\tcornew\AppData\Local\Temp\ipykernel_12076\3996712011.py:23: RuntimeWarning: overflow encountered in exp
  mu_away = np.exp(alpha[j_vals] - beta[i_vals])
c:\Users\tcornew\AppData\Local\anaconda3\Lib\site-packages\scipy\stats\_discrete_distns.py:860: RuntimeWarning: invalid value encountered in subtract
  Pk = special.xlogy(k, mu) - gamln(k + 1) - mu



==== Optimization Results ====
Optimization converged successfully.
delta (home advantage): 0.0245
mean alpha: 0.0000, mean beta: -4.5826
Finished initialization


In [24]:
forecast_results_poisson_no_intercept, latent_history_poisson_no_intercept = run_gas_forecast_loop(
    train_test_data=train_test_data,
    test_data=test_data,
    all_teams=all_teams,
    f1_estimate=f1_estimate_poisson_no_intercept,
    model_class=GASPoisson,
    compute_probs_fn=compute_toto_probs,
    distr='poisson'
)


=== Training for season 2018/2019 ===


c:\Users\tcornew\Downloads\thesis\code\m3\final\final\functions.py:456: RuntimeWarning: divide by zero encountered in log
  return x * np.log(mu) - mu - gammaln(x + 1)
c:\Users\tcornew\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0



==== Optimization Results ====
a1=0.0005, a2=0.0005
b1=0.9950, b2=0.9950
delta=0.0350
Optimization converged successfully.

=== Forecasting for season 2018/2019 ===

=== Training for season 2019/2020 ===

==== Optimization Results ====
a1=0.0005, a2=0.0005
b1=0.9950, b2=0.9950
delta=0.0350
Optimization converged successfully.

=== Forecasting for season 2019/2020 ===

=== Training for season 2020/2021 ===

==== Optimization Results ====
a1=0.0004, a2=0.0005
b1=0.9960, b2=0.9957
delta=0.0272
Optimization converged successfully.

=== Forecasting for season 2020/2021 ===

=== Training for season 2021/2022 ===

==== Optimization Results ====
a1=0.0004, a2=0.0004
b1=0.9971, b2=0.9966
delta=0.0245
Optimization converged successfully.

=== Forecasting for season 2021/2022 ===

=== Training for season 2022/2023 ===

==== Optimization Results ====
a1=0.0004, a2=0.0005
b1=0.9981, b2=0.9956
delta=0.0232
Optimization converged successfully.

=== Forecasting for season 2022/2023 ===
Finished forec

In [25]:
forecast_df_poisson_no_intercept = pd.DataFrame(forecast_results_poisson_no_intercept)

## Log scores

In [26]:
log_score_forecast_df_nb_static = forecast_df_nb_static["log_score"].mean()
log_score_forecast_df_nb = forecast_df_nb["log_score"].mean()
log_score_forecast_df_nb_no_intercept = forecast_df_nb_no_intercept["log_score"].mean()
log_score_forecast_df_poisson_static = forecast_df_poisson_static["log_score"].mean()
log_score_forecast_df_poisson = forecast_df_poisson["log_score"].mean()
log_score_forecast_df_poisson_no_intercept = forecast_df_poisson_no_intercept["log_score"].mean()


print(f"Log Score (Negative Binomial static): {log_score_forecast_df_nb_static:.3f}")
print(f"Log Score (Negative Binomial with intercept): {log_score_forecast_df_nb:.3f}")
print(f"Log Score (Negative Binomial without intercept): {log_score_forecast_df_nb_no_intercept:.3f}")
print(f"Log Score (Poisson static): {log_score_forecast_df_poisson_static:.3f}")
print(f"Log Score (Poisson with intercept): {log_score_forecast_df_poisson:.3f}")
print(f"Log Score (Poisson without intercept): {log_score_forecast_df_poisson_no_intercept:.3f}")

Log Score (Negative Binomial static): 0.682
Log Score (Negative Binomial with intercept): 0.636
Log Score (Negative Binomial without intercept): 0.636
Log Score (Poisson static): 0.684
Log Score (Poisson with intercept): 0.636
Log Score (Poisson without intercept): 0.637


## ARPS per round

In [27]:
rps_nb_static_by_round = forecast_df_nb_static.groupby("date")["rps"].mean()
rps_nb_by_round = forecast_df_nb.groupby("date")["rps"].mean()
rps_nb_no_intercept_by_round = forecast_df_nb_no_intercept.groupby("date")["rps"].mean()
rps_poisson_static_by_round = forecast_df_poisson_static.groupby("date")["rps"].mean()
rps_poisson_by_round = forecast_df_poisson.groupby("date")["rps"].mean()
rps_poisson_no_intercept_by_round = forecast_df_poisson_no_intercept.groupby("date")["rps"].mean()

common_dates_nb_static = rps_nb_by_round.index.intersection(rps_nb_static_by_round.index)
common_dates_nb_no_intercept = rps_nb_by_round.index.intersection(rps_nb_no_intercept_by_round.index)
common_dates_poisson_static = rps_nb_by_round.index.intersection(rps_poisson_static_by_round.index)
common_dates_poisson = rps_nb_by_round.index.intersection(rps_poisson_by_round.index)
common_dates_poisson_no_intercept = rps_nb_by_round.index.intersection(rps_poisson_no_intercept_by_round.index)

loss_nb_static = rps_nb_static_by_round[common_dates_nb_static].values  # candidate
loss_nb = rps_nb_by_round[common_dates_poisson].values  # benchmark
loss_nb_no_intercept = rps_nb_no_intercept_by_round[common_dates_nb_no_intercept].values  # candidate
loss_poisson_static = rps_poisson_static_by_round[common_dates_poisson_static].values  # candidate
loss_poisson = rps_poisson_by_round[common_dates_poisson].values  # candidate
loss_poisson_no_intercept = rps_poisson_no_intercept_by_round[common_dates_poisson_no_intercept].values  # candidate

In [28]:
arps_nb_static = loss_nb_static.mean()
arps_nb = loss_nb.mean()
arps_nb_no_intercept = loss_nb_no_intercept.mean()
arps_poisson_static = loss_poisson_static.mean()
arps_poisson = loss_poisson.mean()
arps_poisson_no_intercept = loss_poisson_no_intercept.mean()

print(f"ARPS (Negative Binomial static): {arps_nb_static:.5f}")
print(f"ARPS (Negative Binomial with intercept): {arps_nb:.5f}")
print(f"ARPS (Negative Binomial without intercept): {arps_nb_no_intercept:.5f}")
print(f"ARPS (Poisson static): {arps_poisson_static:.5f}")
print(f"ARPS (Poisson with intercept): {arps_poisson:.5f}")
print(f"ARPS (Poisson without intercept): {arps_poisson_no_intercept:.5f}")

ARPS (Negative Binomial static): 0.12030
ARPS (Negative Binomial with intercept): 0.11123
ARPS (Negative Binomial without intercept): 0.11138
ARPS (Poisson static): 0.12061
ARPS (Poisson with intercept): 0.11125
ARPS (Poisson without intercept): 0.11145


## DM statistics

In [29]:
dm_stat, p_val = diebold_mariano(loss_nb, loss_nb_static)

print("Diebold-Mariano Test on per-round RPS (NB with intercept vs NB static):")
print(f"DM statistic: {dm_stat:.2f}")
print(f"p-value     : {p_val:.2f}")

Diebold-Mariano Test on per-round RPS (NB with intercept vs NB static):
DM statistic: 9.26
p-value     : 0.00


In [30]:
dm_stat, p_val = diebold_mariano(loss_nb, loss_nb_no_intercept)

print("Diebold-Mariano Test on per-round RPS (NB with intercept vs NB without intercept):")
print(f"DM statistic: {dm_stat:.2f}")
print(f"p-value     : {p_val:.2f}")

Diebold-Mariano Test on per-round RPS (NB with intercept vs NB without intercept):
DM statistic: 2.49
p-value     : 0.01


In [31]:
dm_stat, p_val = diebold_mariano(loss_nb, loss_poisson_static)

print("Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson static):")
print(f"DM statistic: {dm_stat:.2f}")
print(f"p-value     : {p_val:.2f}")

Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson static):
DM statistic: 9.41
p-value     : 0.00


In [32]:
dm_stat, p_val = diebold_mariano(loss_nb, loss_poisson_no_intercept)

print("Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson without intercept):")
print(f"DM statistic: {dm_stat:.2f}")
print(f"p-value     : {p_val:.2f}")

Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson without intercept):
DM statistic: 1.03
p-value     : 0.30


In [33]:
dm_stat, p_val = diebold_mariano(loss_nb, loss_poisson)

print("Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson with intercept):")
print(f"DM statistic: {dm_stat:.2f}")
print(f"p-value     : {p_val:.2f}")

Diebold-Mariano Test on per-round RPS (NB with intercept vs Poisson with intercept):
DM statistic: 0.71
p-value     : 0.48
